In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.compute as pc

---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons

In [5]:
# create instance of capability
kn = Knowledge.from_memory()

In [6]:
text = ('You took too long. You are not easy to deal with.\n\nPayment Failure/Incorrect Payment.\n\nYou provided '
        'me with incorrect information. Unhappy with delay.\n\nUnsuitable advice. You never answered my question.\n\n'
        'You did not understand my needs.\n\nI have been mis-sold. My details are not accurate.')
arr = pa.array([text], pa.string())
tbl = pa.table([arr], names=['text'])
result = kn.tools.text_to_paragraphs(tbl)

building paragraphs:   0%|          | 0/62 [00:00<?, ?it/s]

paragraph stats:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
model = SentenceTransformer('multi-qa-mpnet-base-cos-v1', truncate_dim=512)
model